In [ ]:
import scipy.io
import numpy as np

In [ ]:
mat_data = scipy.io.loadmat('./data_preprocessed_matlab/s02.mat') 
original_data = mat_data['data']
original_label = mat_data['labels']

# (40, 40, 8064) 40试验，40通道（前32为eeg），63s（前3秒没用） x 128Hz = 8064
print("original_data形状：", original_data.shape)
# (40, 4) 40试验，4个评价指标
print("original_label形状：", original_label.shape)

In [ ]:
'''
    处理eeg，从(40, 40, 8064)变成(800, 32, 384)
'''
# 前32为eeg，去除前3秒
sliced_data = original_data[:, :32, 384:] 
print("sliced_data形状：", sliced_data.shape) # (40, 32, 7680)

# 将数据切片成 3 秒一段
segment_size = 3 * 128
num_segments = sliced_data.shape[2] // segment_size # 7680/3/128 = 20
segments_array = np.zeros((sliced_data.shape[0], sliced_data.shape[1], num_segments, segment_size))

# 将数据按照每 384 个元素一段切开，并存储到 segments_array 中
for i in range(num_segments):
    start_index = i * segment_size
    end_index = (i + 1) * segment_size
    segments_array[:, :, i, :] = sliced_data[:, :, start_index:end_index]
# (40, 32, 20, 384)
print("segments_array形状：", segments_array.shape)

# 使用 transpose 函数交换第二和第三维度
reshaped_segments_array = segments_array.transpose(2, 0, 1, 3)
# (20, 40, 32, 384)
print("重新排列后的数组形状：", reshaped_segments_array.shape)

# 使用 reshape 函数合并前两个维度
merged_segments_array = reshaped_segments_array.reshape(-1, reshaped_segments_array.shape[2], reshaped_segments_array.shape[3])
# (800, 32, 384)
print("合并后的数组形状：", merged_segments_array.shape)
eeg_data = merged_segments_array

In [ ]:
'''
    处理labels，从(40, 4)变成(800,)
'''
valence = original_label[:,0]
arousal = original_label[:,1]
# HAHV--1, LAHV--2, LALV--3, HALV--4 完全按照象限来分
VA_labels = np.where((valence > 5) & (arousal > 5), 1,
          np.where((valence >= 5) & (arousal < 5), 2,
            np.where((valence < 5) & (arousal < 5), 3, 4)))
print("V:", valence)
print("A:", arousal)
print(VA_labels)

# 将 VA_labels 从 40 扩展到 800，VA_labels中的一个数复制20次
expanded_VA_labels = np.repeat(VA_labels, num_segments)
print(expanded_VA_labels.shape) # (800,)
labels = expanded_VA_labels 

In [ ]:
import pandas as pd
import mne
from mne import io
from mne.datasets import sample
from mne.preprocessing import ICA
import os

sfreq = 128  # 采样率为128Hz
channels = 32
samples = 384
num_trials = 800
ch_names = ['Fp1', 'AF3', 'F7', 'F3', 'FC1', 'FC5', 'T7', 'C3', 'CP1', 'CP5', 'P7', 'P3', 
            'Pz', 'PO3', 'O1', 'Oz', 'O2', 'PO4', 'P4', 'P8', 'CP6', 'CP2', 'C4', 'T8', 
            'FC6', 'FC2', 'F4', 'F8', 'AF4', 'FP2', 'Fz', 'Cz']
ch_types = ['eeg'] * channels

# eeg_data_file = os.path.join(eeg_data_folder, 's01_eeg.npy')
# label_file = os.path.join(eeg_data_folder, 's01_labels.npy')
# eeg_data = np.load(eeg_data_file)
# labels = np.load(label_file)

data_list = []
# 创建MNE的Raw对象
info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sfreq)
eeg_data_segments = np.split(eeg_data, num_trials, axis=0)
for segment in eeg_data_segments:
    segment_2d = segment.reshape(-1, channels).T
    print("wzt segment_2d", segment_2d.shape) # (32, 384)
    # 创建Raw对象
    raw = mne.io.RawArray(segment_2d, info=info)
    
    # 滤波
    raw.filter(l_freq=1.0, h_freq=50.0)
    
    # 创建ICA对象并拟合数据
    ica = ICA(n_components=channels, random_state=0, max_iter=1000)  # 调整参数
    ica.fit(raw)
    
    # 应用ICA滤波
    ica.exclude = []  # 将排除的独立成分列表设置为空
    ica.apply(raw)
    
    # 将滤波后的数据添加到列表
    data_list.append(raw.get_data().T)

# 将数据列表转换为3D数组
eeg_data_filtered = np.array(data_list)
print("wzt eeg_data_filtered", eeg_data_filtered.shape)

In [ ]:
'''
    保存数据
'''
np.save('./EEGData/s02_eeg.npy', eeg_data) # (800, 32, 384)
np.save('./EEGData/s02_labels.npy', labels) # (800,)